In [1]:
import openai
import pandas as pd

In [2]:
with open('01.txt', 'r') as fh:
    tmp = fh.read()
    itemlist = tmp.split(',')

In [3]:
itemlist = str(itemlist)

In [5]:
keyfile = open("key.txt", "r")
key = keyfile.readline()
openai.api_key = key

In [6]:
data = [itemlist[0:3200], itemlist[3201:6400], 
        itemlist[6401:9600], itemlist[9601:12800], itemlist[12801:16000]]

In [11]:
data[0]

"['\\nBlockchain in education management: present and future applications \\nPreeti Bhaskar \\nUniversity of Technology and Applied Sciences', ' Ibra', ' Oman and ICFAI Business School', ' ICFAI University', ' Dehradun', ' India \\n\\nChandan Kumar Tiwari \\nSymbiosis Centre for Management Studies', ' Noida', ' India and Symbiosis International (Deemed University)', ' Pune', ' India', ' and \\n\\nAmit Joshi \\nICFAI Business School', ' ICFAI University', ' Dehradun', ' India \\nAbstract \\nPurpose – This paper aims to provide a systematic literature review on blockchain technology in education to offer a detailed understanding of the present scenario in terms of bene.ts', ' barriers', ' present blockchain technology application and future areas where blockchain technology can be implemented in the other .elds of education. \\nDesign/methodology/approach – A bibliometric analysis is conducted on for data in the publications', ' journals', ' authors and citations were collected', ' and e

In [12]:
def chatgptfn(sub_list):
    result = ''
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are an assistant"},
            {"role": "user", "content": f"{sub_list} :give me a summary"}
        ]
    )
    for choice in response.choices:
        result += choice.message.content
    return result

In [13]:
data[0] = chatgptfn(data[0])

In [14]:
data[0]

'This paper provides a literature review on the application of blockchain technology in education. The study examines the benefits and barriers associated with blockchain technology, as well as its current applications in the educational sector. Additionally, the paper identifies potential future areas in which blockchain technology can be implemented. The research is based on bibliometric analysis and provides a foundation for educators, policymakers, and researchers to explore the potential of blockchain technology in education.'

In [15]:
data[1] = chatgptfn(data[1])

In [9]:
data[2] = chatgptfn(data[2])
data[3] = chatgptfn(data[3])

In [18]:
for i in range(0,5):
    data[i] = chatgptfn(data[i])

In [19]:
import gensim
from gensim import corpora
from pprint import pprint

In [20]:
# 創建詞袋
texts = [[word for word in document.lower().split()] for document in data]
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

In [21]:
# 訓練 LDA 模型
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, 
                                            num_topics=3, random_state=100, update_every=1, 
                                            chunksize=100, passes=10, alpha='auto', per_word_topics=True)

In [22]:
# 輸出主題模型分析結果
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)
pyLDAvis.display(vis)

C:\Users\pecu6\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
